<b> Coding by python2.7
   
authors: Qixiang PENG, Zizhao LI

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB

<b>load data

In [2]:
train = pd.read_csv('./data/train.csv')
# class between [1,9]
train_y = train['target'].apply(lambda s: int(s[-1:])).values
train_X = train.drop('id', axis=1)
train_X = train_X.drop('target', axis=1)
X_test = pd.read_csv('./data/test.csv')
X_test = X_test.drop('id', axis=1)
# split train set into 2 parts with same distribution: 80% train, 20% validation
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_index, test_index in sss.split(train_X.values, train_y):
    X_train = train_X.values[train_index]
    X_val = train_X.values[test_index]

    y_train = train_y[train_index]
    y_val = train_y[test_index]

<b>use original NB

In [7]:
NB = MultinomialNB()
NB.fit(X_train, y_train)
pred= NB.predict_proba(X_val)
score = log_loss(y_val, pred)
print score

3.54678781393


<b> use MultinomialNB with Calibrated prob

In [8]:
NB = MultinomialNB()
NB_isotonic = CalibratedClassifierCV(NB, cv=5, method='isotonic')
NB_isotonic.fit(X_train, y_train)
prob_pos_isotonic = NB_isotonic.predict_proba(X_val)
NB_score_isotonic = log_loss(y_val, prob_pos_isotonic)
print NB_score_isotonic

0.808216645041


<b> export 

In [9]:
result = NB_isotonic.predict_proba(X_test)
columns = ["id","Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"]
rlt = np.zeros(result.shape[0] * 9).reshape((result.shape[0],9)).astype(float)

i = 0
for class_i in result:
    rlt[i] = class_i
    i += 1
    
r = []
i = 1
for class_i in rlt:
    p = [i] + list(map(str, class_i.tolist()))
    i += 1
    r.append(p)
r = np.array(r)

out = pd.DataFrame(r,columns = columns)
out.to_csv('result_NB.csv',index = False)